In [1]:
import os
import sys
import matplotlib.pyplot as plt
import glob
import random
import pandas as pd
import easyaccess as ea
import numpy as np
import re
import psycopg2
import subprocess
from subprocess import Popen, PIPE
import argparse
from multiprocessing import Pool,Process, Queue, current_process, Semaphore
import time
import queue
import functions_imageproc

In [2]:
def EXPlist(explist):
    
    file_to_read = open(explist,'r')
    exp_file = file_to_read.readlines()
    elist = [exp.strip() for exp in exp_file]
    file_to_read.close()

    return elist

In [3]:
inputted_exp_list = input("Please input the filepath to your exp.list file")

Please input the filepath to your exp.list fileexposurestest.list


In [4]:
# filepath = 'test_exp_list.list'
filepath = inputted_exp_list
sample_exp_set = EXPlist(filepath)
number_exps = len(sample_exp_set)

In [5]:
#initializing information for the processing function arguments
number_of_tasks = number_exps
#number of processes is how many to submit/run at one time 
number_of_processes = 5
exp_set = sample_exp_set

In [6]:
def main():
    
    #initialize queues so the system can recognize what's been done and what needs to be done 
    exps_to_run = Queue()
    finished_exps = Queue()
    processes = []
    sema = Semaphore(number_of_processes)

    for i in range(number_of_tasks):
        exps_to_run.put(int(i))

    # creating processes
    for w in range(number_of_processes):
        sema.acquire()
        p = Process(target=functions_imageproc.run_dagsh, args=(exps_to_run, finished_exps, exp_set))
        processes.append(p)
        p.start()

    # completing processes
    for p in processes:
        p.join()

    # print what has been done to double check everything went smoothly
    while not finished_exps.empty():
        print(finished_exps.get())

    return True


if __name__ == '__main__':
    main()

Running ./DAGMaker.sh 938820
Running ./DAGMaker.sh 938821
Running ./DAGMaker.sh 938822
Running ./DAGMaker.sh 938823
Running ./DAGMaker.sh 938824
All done with ./DAGMaker.sh 938824
All done with ./DAGMaker.sh 938821
All done with ./DAGMaker.sh 938823
All done with ./DAGMaker.sh 938822
938824 is done by Process-5
938820 is done by Process-1
938821 is done by Process-2
938823 is done by Process-4
938822 is done by Process-3


In [ ]:
#get the band for each of these exposures [exp band, exp band, exp band, exp band] and the nite and season 